In [1]:
%pylab inline
import pandas as pd

import seaborn as sns


Populating the interactive namespace from numpy and matplotlib


In [4]:

D= pd.read_excel('../raw_data/vitamin_D.xlsx').ffill()

#D.index= pd.MultiIndex.from_frame(D[['Country','Reference']])
D['VitaminD']=D['25(OH)D (nmol/L)a'].map(lambda s: float(str(s).replace(' (+)','')))
D.Representative=D.Representative.map({'Yes':True,'No':False,'NA':np.nan})


In [5]:
D.query('Reference=="[108]"')

,Country,Reference,Representative,Age (years),Sex,N,25(OH)D (nmol/L)a,Season,Study colour codeb,Country colour codeb,Country colour code rationale,VitaminD
54,Greece,[108],False,60–89,M + F,279.0,42.9,Mixed,ORANGE,ORANGE,Based on a single study and on a weighted average,42.9
55,Greece,[108],False,19–46,M + F,44.0,85.7,Mixed,GREEN,ORANGE,Based on a single study and on a weighted average,85.7


In [6]:
D.loc[D[['Country','Reference']].duplicated(keep=False) ]

,Country,Reference,Representative,Age (years),Sex,N,25(OH)D (nmol/L)a,Season,Study colour codeb,Country colour codeb,Country colour code rationale,VitaminD
9,Belgium,[71],False,20+,M,270.0,71.5,Winter,YELLOW,YELLOW,Based on a weighted average,71.5
10,Belgium,[71],False,20+,F,272.0,73.5,Winter,YELLOW,YELLOW,Based on a weighted average,73.5
34,Finland,[94],False,30–97,M,2736.0,45.1,Mixed,ORANGE,ORANGE,"Representative of the entire country, populati...",45.1
35,Finland,[94],False,30–97,F,3299.0,45.2,Mixed,ORANGE,ORANGE,"Representative of the entire country, populati...",45.2
38,Finland,[97],False,31–43,M,126.0,45,Mixed,ORANGE,ORANGE,"Representative of the entire country, populati...",45.0
...,...,...,...,...,...,...,...,...,...,...,...,...
217,USA,[223],False,55+,F,20.0,85,Mixed,GREEN,YELLOW,"Representative of the entire country, populati...",85.0
221,USA,[227],False,64–92,M,142.0,67.5,Mixed,YELLOW,YELLOW,"Representative of the entire country, populati...",67.5
222,USA,[227],False,64–93,F,195.0,57.7,Mixed,YELLOW,YELLOW,"Representative of the entire country, populati...",57.7
225,Vietnam,[230],True,18–87,M,205.0,92,Mixed,GREEN,GREEN,Population-based data for a region/city of the...,92.0


In [7]:
from IPython.display import display

In [8]:
def weighted_average(data):
    return data.eval('N*VitaminD').mean()/data.N.sum()

In [9]:
Averages={}

for country,dc in  D.groupby('Country'):
    
    if dc.Representative.any():
        dc=dc.loc[dc.Representative]    
    
    # collaps studies with M and F seperated 
    for ref,subd in dc.groupby('Reference'):
        if subd.shape[0]>1:

            if  (subd.iloc[0]['Age (years)']==subd['Age (years)']).all():
                
                assert ((subd.Sex=='M').sum()==1) & ((subd.Sex=='F').sum()==1), display(subd)
                assert subd.iloc[0]['Age (years)']== subd.iloc[1]['Age (years)'],display(subd)

                dc.drop(subd.index[0],inplace=True)

                dc.loc[subd.index[0],'Sex'] = 'M + F'
                dc.loc[subd.index[0],'N'] = subd.N.sum()
                dc.loc[subd.index[0],'VitaminD'] = weighted_average(subd)
            #else:
             #   print(country)
              #  print(ref)

               # display(subd)
    
    
    
    if (dc.Sex=='M + F').any():
        dc=dc.query("Sex=='M + F'")

    if dc.shape[0]==1:
        Averages[country]= dc.VitaminD.iloc[0]
    else:
        
        if not (dc['Country colour code rationale'].str.contains('weighted average')).all():
            
            
            print(country)
            print(dc['Country colour code rationale'].iloc[0])
            display(dc)
            
        
        Averages[country]= weighted_average(dc)
        

/Users/silas/miniconda3/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/Users/silas/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Spain
Representative of a region/city of the country, population-based


,Country,Reference,Representative,Age (years),Sex,N,25(OH)D (nmol/L)a,Season,Study colour codeb,Country colour codeb,Country colour code rationale,VitaminD
132,Spain,[161],True,15–70,F,127.0,49.9,Mixed,ORANGE,YELLOW,Representative of a region/city of the country...,49.900000
131,NaN,NaN,NaN,NaN,M + F,253.0,NaN,NaN,NaN,NaN,NaN,25.647233


Thailand
Representative of a region/city of the country, population-based


/Users/silas/miniconda3/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/Users/silas/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


...

United Kingdom
Representative of the entire country, population-based


/Users/silas/miniconda3/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/Users/silas/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Country,Reference,Representative,Age (years),Sex,N,25(OH)D (nmol/L)a,Season,Study colour codeb,Country colour codeb,Country colour code rationale,VitaminD
157,United Kingdom,[180],True,65+,F,320.0,48.4,Mixed,ORANGE,YELLOW,"Representative of the entire country, populati...",48.400000
159,United Kingdom,[181],True,65+,F,1120.0,48,Mixed,ORANGE,YELLOW,"Representative of the entire country, populati...",48.000000
156,NaN,NaN,NaN,NaN,M + F,642.0,NaN,NaN,NaN,NaN,NaN,26.156075
158,NaN,NaN,NaN,NaN,M + F,2070.0,NaN,NaN,NaN,NaN,NaN,25.147343


Vietnam
Population-based data for a region/city of the country


,Country,Reference,Representative,Age (years),Sex,N,25(OH)D (nmol/L)a,Season,Study colour codeb,Country colour codeb,Country colour code rationale,VitaminD
226,Vietnam,[230],True,18–87,F,432.0,75.3,Mixed,GREEN,GREEN,Population-based data for a region/city of the...,75.300000
225,NaN,NaN,NaN,NaN,M + F,637.0,NaN,NaN,NaN,NaN,NaN,40.337206


In [10]:
Averages= pd.Series(Averages,name='Vitamin D')

In [11]:
Averages.to_csv('../data/VitaminD_per_country.tsv',sep='\t',header=True)